# Prévision du prix de l’électricité – Notebook de l’expert

Ce notebook présente une solution complète pour le ChallengeData d’Elmy portant sur la prédiction du prix de l’électricité. Nous détaillons chaque étape du processus : chargement des données, analyse exploratoire, préparation du jeu de données, entraînement d’un modèle de classification (XGBoost) et génération des prédictions. Le code est commenté en français et modulaire, prêt à l’emploi dans un Jupyter Notebook classique. Les bibliothèques utilisées incluent pandas, numpy, seaborn, matplotlib, xgboost, scikit-learn et openpyxl.

## 1. Chargement des données

On commence par importer les bibliothèques nécessaires et charger les fichiers Excel fournis. Ici, openpyxl est spécifié comme moteur pour read_excel, afin d’assurer la compatibilité avec les fichiers .xlsx. Nous vérifions ensuite la taille des données chargées (nombre de lignes et colonnes) pour chaque jeu de données (X_train, Y_train, X_test, Y_random).


In [ ]:
# Import des bibliothèques principales

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# Chargement des fichiers Excel (X_train, Y_train, X_test, Y_random)

X_train = pd.read_excel('C:\Users\julie\OneDrive - CentraleSupelec\casablanca\COURS\ML\PROJET LIL\X_train_Wwou3IE.csv', engine='openpyxl')
Y_train = pd.read_excel('C:\Users\julie\OneDrive - CentraleSupelec\casablanca\COURS\ML\PROJET LIL\y_train_jJtXgMX.csv', engine='openpyxl')
X_test = pd.read_excel('C:\Users\julie\OneDrive - CentraleSupelec\casablanca\COURS\ML\PROJET LIL\X_test_GgyECq8.csv', engine='openpyxl')
Y_random = pd.read_excel('C:\Users\julie\OneDrive - CentraleSupelec\casablanca\COURS\ML\PROJET LIL\y_random_pt8afo8.csv', engine='openpyxl')

# Aperçu des dimensions des jeux de données
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_random shape:", Y_random.shape)


## 2. Analyse exploratoire des données
Nous explorons les données pour mieux les comprendre. On examine les types de variables, la présence de valeurs manquantes, et des statistiques descriptives. On regarde également les corrélations entre variables numériques. Enfin, on étudie la distribution de la variable cible d’origine spot_id_delta dans Y_train.

### Aperçu des types et de l'échantillon de données

In [ ]:
print("Types de variables X_train :\n", X_train.dtypes)
print("\nAperçu de X_train :")
display(X_train.head())

print("\nTypes de variables Y_train :\n", Y_train.dtypes)
print("\nAperçu de Y_train :")
display(Y_train.head())

### Recherche de valeurs manquantes dans X_train

In [ ]:
missing_values = X_train.isnull().sum()
print("\nValeurs manquantes par colonne dans X_train :")
print(missing_values[missing_values > 0])

### Statistiques descriptives sur X_train

In [ ]:
print("\nStatistiques descriptives de X_train :")
display(X_train.describe())

### Corrélation entre variables numériques (X_train + Y_train)


In [ ]:
data_concat = pd.concat([X_train, Y_train], axis=1)
corr_matrix = data_concat.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Matrice de corrélation (variables numériques + cible)")
plt.show()

Ensuite, on examine spécifiquement la distribution de la variable cible d’origine spot_id_delta, afin de voir par exemple si elle suit une loi normale ou si elle est centrée autour de zéro.

### Distribution de la variable cible continue spot_id_delta


In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(Y_train['spot_id_delta'], bins=50, kde=True)
plt.title("Distribution de spot_id_delta dans Y_train")
plt.xlabel("spot_id_delta")
plt.ylabel("Fréquence")
plt.show()

## 3. Transformation de la cible en variable binaire

La variable cible spot_id_delta est transformée en variable binaire spot_id_delta_binary : elle vaut 1 si spot_id_delta est positif (supérieur à 0), et 0 sinon (négatif ou zéro). On ajoute cette colonne à Y_train et vérifie l’équilibre des classes.

### Création d'une cible binaire : 1 si spot_id_delta > 0, 0 sinon


In [ ]:
Y_train['spot_id_delta_binary'] = (Y_train['spot_id_delta'] > 0).astype(int)


### Vérification de la répartition des classes


In [ ]:
counts = Y_train['spot_id_delta_binary'].value_counts()
print("Répartition des classes (0 vs 1) :")
print(counts)
plt.figure(figsize=(4,4))
sns.countplot(x='spot_id_delta_binary', data=Y_train)
plt.title("Nombre de cas 0 et 1 dans la cible binaire")
plt.xlabel("spot_id_delta_binary")
plt.ylabel("Nombre d'exemples")
plt.show()


Cette transformation fait de notre problème un problème de classification binaire. On constate ainsi la proportion d’exemples positifs et négatifs.

## 4. Prétraitement des données

### 4.1 Encodage de la date

La colonne DELIVERY_START contient une date/heure. Nous convertissons cette colonne au format datetime et en extrayons des caractéristiques utiles, comme l’heure de la journée (hour) et le jour de la semaine (dayofweek). Nous pourrons ainsi intégrer l’effet temporel dans le modèle. Après extraction, on peut supprimer la colonne originale DELIVERY_START si elle n’apporte pas d’information supplémentaire.

In [ ]:
def encoder_dates(df):
    """
    Convertit la colonne DELIVERY_START en datetime et en extrait les composantes heure et jour de la semaine.
    La colonne originale DELIVERY_START est ensuite supprimée.
    """
    # Conversion en datetime
    df['DELIVERY_START'] = pd.to_datetime(df['DELIVERY_START'])
    # Extraction de l'heure et du jour de la semaine (0 = lundi)
    df['hour'] = df['DELIVERY_START'].dt.hour
    df['day_of_week'] = df['DELIVERY_START'].dt.dayofweek
    # Suppression de la colonne initiale
    df.drop(columns=['DELIVERY_START'], inplace=True)
    return df

#### Application de l'encodage sur X_train et X_test


In [ ]:
X_train = encoder_dates(X_train)
X_test = encoder_dates(X_test)
print("Colonnes X_train après encodage des dates :", X_train.columns.tolist())


### 4.2 Autres prétraitements éventuels

On vérifie à nouveau la présence éventuelle de valeurs manquantes après encodage et on les traite (par exemple, on pourrait les imputer ou simplement retirer les lignes manquantes). Pour ce notebook, nous supposons qu’il n’y a pas de valeurs manquantes critiques après cette étape, ou bien qu’elles sont peu nombreuses. Si nécessaire, on pourrait imputer des valeurs ou supprimer des colonnes entières.

#### Vérification des valeurs manquantes après prétraitement


In [ ]:
missing_train = X_train.isnull().sum().sum()
missing_test = X_test.isnull().sum().sum()
print(f"Nombre total de valeurs manquantes dans X_train (après prétraitement) : {missing_train}")
print(f"Nombre total de valeurs manquantes dans X_test (après prétraitement) : {missing_test}")

Si on trouvait des valeurs manquantes importantes, on les traiterait ici, par exemple avec :



In [ ]:
# Exemple : imputation par la médiane pour les colonnes numériques (non exécuté si pas de NaN)

# X_train.fillna(X_train.median(), inplace=True)
# X_test.fillna(X_train.median(), inplace=True)  # on utilise la médiane du train

### 4.3 Séparation en jeu d’entraînement et de validation

Nous séparons ensuite le jeu X_train prétraité et la cible binaire Y_train['spot_id_delta_binary'] en un sous-jeu d’entraînement (X_train_sub, y_train_sub) et un sous-jeu de validation (X_val, y_val). Cela nous permettra d’évaluer le modèle sur des données non vues pendant l’entraînement. Ici, on utilise par exemple 80% pour l’entraînement et 20% pour la validation.

#### Séparation des données en train/validation


In [ ]:
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, Y_train['spot_id_delta_binary'], test_size=0.2, random_state=42
)

print("Forme de X_train_sub :", X_train_sub.shape)
print("Forme de X_val :", X_val.shape)
print("Forme de y_train_sub :", y_train_sub.shape)
print("Forme de y_val :", y_val.shape)

## 5. Entraînement du modèle de classification (XGBoost)

Nous entraînons un modèle de XGBoost pour la classification binaire. XGBoost est bien adapté aux données tabulaires et gère les interactions entre les variables. On crée un classifieur XGBClassifier de base et on l’ajuste sur nos données d’entraînement. On peut spécifier random_state pour la reproductibilité et éventuellement régler d’autres hyperparamètres (nombre d’arbres, profondeur, etc.), mais nous prenons les valeurs par défaut pour cette démonstration.

#### Création et entraînement du classifieur XGBoost


In [ ]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_clf.fit(X_train_sub, y_train_sub)

#### Prédiction sur le jeu de validation


In [ ]:
y_pred_val = xgb_clf.predict(X_val)
print("Prédiction effectuée sur le jeu de validation.")

## 6. Évaluation avec la métrique Weighted Accuracy

La métrique d’évaluation du challenge est la Weighted Accuracy. Nous définissons une fonction personnalisée weighted_accuracy qui calcule ce score. Par convention, on peut pondérer davantage la classe positive (valeur 1) pour compenser un éventuel déséquilibre. Par exemple, on utilise ici un poids w=5 sur la classe positive :
$$
\text{Weighted Accuracy} = \frac{\text{WTP} + \text{WTN}}{\text{WTP} + \text{WTN} + \text{WFP} + \text{WFN}}
$$

avec :
- WTP = somme des $|y_i|$ pour les vrais positifs,
- WTN = somme des $|y_i|$ pour les vrais négatifs,
- WFP = somme des $|y_i|$ pour les faux positifs,
- WFN = somme des $|y_i|$ pour les faux négatifs. est vraie et 0 sinon,



In [ ]:
def weighted_accuracy(y_true, y_pred, weight=5):
    """
    Calcule la Weighted Accuracy :
    (w * TP + TN) / (w * (TP + FN) + (TN + FP))
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    wa = (weight * tp + tn) / (weight * (tp + fn) + (tn + fp))
    return wa

#### Calcul du score sur la validation


In [ ]:
wa_score = weighted_accuracy(y_val, y_pred_val, weight=5)
print(f"Weighted Accuracy sur le jeu de validation : {wa_score:.4f}")

On affiche également la matrice de confusion et d’autres métriques classiques pour comprendre les performances :


#### Matrice de confusion


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

tn, fp, fn, tp = confusion_matrix(y_val, y_pred_val).ravel()
print("Matrice de confusion (validation) :")
print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")

#### Rapport de classification standard


In [ ]:
print("\nRapport de classification :")
print(classification_report(y_val, y_pred_val, target_names=['Classe 0', 'Classe 1']))
print("Accuracy simple :", accuracy_score(y_val, y_pred_val))

## 7. Prédictions finales sur X_test

Après évaluation satisfaisante, on applique le même prétraitement à X_test (encodage des dates déjà fait) et on utilise le modèle entraîné pour prédire la classe binaire sur les nouvelles données.

#### Prédiction sur X_test (après prétraitement des dates)


In [ ]:

y_test_pred = xgb_clf.predict(X_test)
print("Prédictions finalisées sur X_test.")

## 8. Création du fichier de soumission

Enfin, on prépare le fichier de soumission ma_submission.csv avec deux colonnes : Id et prediction. On utilise la colonne Id de X_test (ou l’index si aucune colonne Id n’est présente) et la prédiction binaire correspondante. On enregistre le fichier au format CSV sans index supplémentaire.

In [ ]:
# Construction du DataFrame de soumission

if 'Id' in X_test.columns:
    submission = pd.DataFrame({
        'Id': X_test['Id'],
        'prediction': y_test_pred
    })
else:
    submission = pd.DataFrame({
        'Id': X_test.index,
        'prediction': y_test_pred
    })

# Sauvegarde au format CSV
submission.to_csv('ma_submission.csv', index=False)
print("Fichier de soumission 'ma_submission.csv' créé avec succès.")


Le fichier ma_submission.csv contient ainsi, pour chaque identifiant, la prédiction binaire (0 ou 1) correspondant à notre modèle.
Ce notebook fournit un pipeline complet, depuis le chargement des données jusqu’à l’enregistrement des prédictions. Les commentaires et les explications détaillées facilitent la compréhension du flux de travail, et le code est structuré en sections claires pour être facilement reproductible dans un environnement Jupyter.